In [190]:
import pandas as pd

results = pd.read_csv("results/PRBO_results.csv")
results = results.rename(columns={"env":"e","elapse":"v"})

In [191]:
results.head(5)

,e,k,v,profit
0,0,2,32,2184.475
1,0,2,32,1621.900
2,0,2,32,1580.575
3,0,2,32,2362.650
4,0,2,32,1643.575


In [192]:
env = 1

subset = results[(results["e"]==env)]
groups = subset.groupby(["k","v"])
groups["profit"].mean().sort_values(ascending=False)

k  v  
2  32     2352.82775
3  128    2323.07875
2  256    2322.47375
4  64     2320.38075
   128    2316.90650
3  32     2305.71150
4  256    2304.12350
3  64     2297.35475
   256    2280.56125
4  32     2267.62000
2  128    2257.24050
   64     2230.67375
Name: profit, dtype: float64

In [193]:
import numpy as np
from scipy import stats

def ksTest(s):
	return stats.kstest(s, 'norm', (s.mean(), s.std())).pvalue

ksresults = groups["profit"].apply(ksTest)

In [194]:
source = ""

for k in np.unique(results["k"]):
    source += str(k) + " & "
    for i,v in enumerate(np.unique(results["v"])):
        kv = (ksresults[(k,v)]).round(2)
        source += str(kv)
        if (i < len(np.unique(results["v"]))-1):
            source += " & "
        else:
            source += " \\\ \hline "
print(source)

2 & 0.96 & 0.95 & 0.92 & 0.89 \\ \hline 3 & 0.41 & 0.47 & 0.54 & 0.89 \\ \hline 4 & 0.61 & 0.88 & 0.85 & 0.26 \\ \hline 


In [195]:
import numpy as np
from scipy.stats import norm

bestk = 2
bestv = 32

def doublezTest(y):
    x = results[(results["k"]==bestk)&(results["v"]==bestv)]["profit"]
    varx = x.var()*(x.shape[0]-1)/x.shape[0] 
    vary = y.var()*(y.shape[0]-1)/y.shape[0] 
    n1 = x.shape[0]
    n2 = y.shape[0]
    z = (x.mean()-y.mean())/np.sqrt(varx/n1+vary/n2)
    p = 1 - norm.cdf(abs(z))
    return p

zTestResult = groups["profit"].apply(doublezTest)

In [196]:
source = ""

paralist = []

for k in np.unique(results["k"]):
    source += str(k) + " & "
    for i,v in enumerate(np.unique(results["v"])):
        kv = (results[(results["k"]==k)&(results["e"]==env)&(results["v"]==v)].mean()["profit"]/1000).round(2)
        if (k, v) == (bestk, bestv):
            source += "\\textbf{%.2f}" % kv
            paralist.append((k, v))
        elif zTestResult[(k,v)]>0.05:
            source += "\\underline{%.2f}" % kv + "_{%.2f}" % zTestResult[(k,v)].round(2)
            paralist.append((k, v))
        else:
            source += str(kv)
        if (i < len(np.unique(results["v"]))-1):
            source += " & "
        else:
            source += " \\\ \hline "
print(source)

2 & \textbf{2.35} & \underline{2.23}_{0.18} & \underline{2.26}_{0.33} & \underline{2.32}_{0.19} \\ \hline 3 & \underline{2.31}_{0.29} & \underline{2.30}_{0.36} & \underline{2.32}_{0.19} & \underline{2.28}_{0.48} \\ \hline 4 & \underline{2.27}_{0.41} & \underline{2.32}_{0.20} & \underline{2.32}_{0.23} & \underline{2.30}_{0.30} \\ \hline 


In [197]:
print(len(paralist),paralist)

12 [(2, 32), (2, 64), (2, 128), (2, 256), (3, 32), (3, 64), (3, 128), (3, 256), (4, 32), (4, 64), (4, 128), (4, 256)]
